# Analyzing global internet patterns

## 📖 Background
In this competition, you'll be exploring a dataset that highlights internet usage for different countries from 2000 to 2023. Your goal is import, clean, analyze and visualize the data in your preferred tool.

The end goal will be a clean, self explanatory, and interactive visualization. By conducting a thorough analysis, you'll dive deeper into how internet usage has changed over time and the countries still widely impacted by lack of internet availability. 


## 💾 Data

#### You have access to the following file, but you can supplement your data with other sources to enrich your analysis. 

### Interet Usage (`internet_usage.csv`)
|   Column name  |   Description | 
|---------------|-----------|
| Country Name | Name of the country |
| Country Code | Countries 3 character country code|
| 2000 | Contains the % of population of individuals using the internet in 2000  |
| 2001 | Contains the % of population of individuals using the internet in 2001  |
| 2002 | Contains the % of population of individuals using the internet in 2002  |
| 2003 | Contains the % of population of individuals using the internet in 2003  |
| .... | ...  |
| 2023 | Contains the % of population of individuals using the internet in 2023  |

**The data can be downloaded from the _Files_ section (_File_ > _Show workbook files_).**

## 💪 Challenge
Use a tool of your choice to create an interesting visual or dashboard that summarizes your analysis! 

Things to consider:
1. Use this Workspace to prepare your data (optional).
2. Stuck on where to start, here's some ideas to get you started: 
    - Visualize interner usage over time, by country 
    - How has internet usage changed over time, are there any patterns emerging? 
    - Consider bringing in other data to supplement your analysis 
3. Create a screenshot of your main dashboard / visuals, and paste in the designated field. 
4. Summarize your findings in an executive summary.

### 1. Preparing internet access data


In [3]:
import pandas as pd
import numpy as np
import missingno as msno

internet_usage = pd.read_csv("data/downloads/internet_usage.csv") 

In [ ]:
# check for missing values
internet_usage.info()

In [ ]:
# there are no 'null values', so replace the '..' with NaN
internet_usage.replace('..', np.nan, inplace=True)

In [ ]:
fig = msno.matrix(internet_usage)

In [ ]:
internet_usage.set_index(['Country Name', 'Country Code'], inplace=True)

# 2023 is missing a lot of data, so drop this column
internet_usage.drop(columns='2023', inplace=True)

# drop rows missing data every year
all_years: list[str] = [str(year) for year in range(2000, 2023)]
internet_usage.dropna(axis=0, how='all', subset=all_years, inplace=True)

# visualise results
msno.matrix(internet_usage)

In [ ]:
# still lots of missing data, so fill from the previous/lead year 
# a maximum of 2 times
# limit area inside to only fill NaNs if surrounded by valid values
internet_usage = internet_usage.T
internet_usage.ffill(limit=2, inplace=True, limit_area='inside')
internet_usage.bfill(limit=2, inplace=True, limit_area='inside')

# visualise results
msno.matrix(internet_usage.T)

In [ ]:
# view data
internet_usage.head()

In [ ]:
# Reverse transposition
internet_usage = internet_usage.T

# Update data types
internet_usage = internet_usage.astype('float')
internet_usage.dtypes

In [ ]:
# Round floating point numbers to 2 d.p.
internet_usage = internet_usage.apply(round, args=([2]))

# Save to csv
internet_usage.to_csv('data/export/internet_usage.csv')

# view data
internet_usage.head()

In [ ]:
# reshape data in long format for racing chart
internet_usage_lf = internet_usage.reset_index().melt(id_vars=['Country Name', 'Country Code'], var_name='Year', value_name='access %')

# save to csv
internet_usage_lf.to_csv('data/export/internet_usage_long.csv')

# view data
internet_usage_lf.head()

#### 1.1 Internet usage by gender

In [6]:
# Code to process world bank data

def load_world_bank_data(file_path: str) -> pd.DataFrame:
    """Load, clean, rename columns
    Params:
    file_path: str: Path to file on drive. Must be to .csv file.
    
    Returns:
    df: pandas DataFrame"""
    index_cols: list[str] = ['Series Name', 'Country Name', 'Country Code']

    # Load the dataframe
    df = pd.read_csv(file_path, index_col=index_cols)
    
    # Remove unneed column
    df.drop('Series Code', axis=1, inplace=True)

    # Remove the last two rows
    df.drop([-1, -2], inplace=True)

    # drop fully missing rows
    df.dropna(how='all', inplace=True)

    # Rename year columns
    df.rename(lambda x: x[:4], axis='columns', inplace=True)
    
    # Reset the index
    df.reset_index(inplace=True)

    return df

gender_usage = load_world_bank_data('data/downloads/access_genders/internet_access_genders.csv')

In [ ]:
# View null value matrix
msno.matrix(gender_usage)

In [9]:
# function export world bank data
def export_world_bank_data(df: pd.DataFrame, file_name) -> None:
    """Save dateframe as csv in wide and long format"""
    directory = 'data/export/'

    # export in wide format (As is)
    df.to_csv(f"data/export/{file_name}.csv")


def transform_to_long(df, id_vars: list[str], var_name: str, value_name: str) -> pd.DataFrame:
    df_long = df.reset_index().melt(id_vars=id_vars, var_name=var_name, value_name=value_name)
    return df_long


In [10]:
# Export gender usage data
gender_usage_long = transform_to_long(gender_usage, id_vars=['Series Name', 'Country Name', 'Country Code'], \
                    var_name='Year', value_name='% of pop')

export_world_bank_data(gender_usage, 'gender_usage')
export_world_bank_data(gender_usage_long, 'gender_usage_long')

### 2. Subscription type

In [11]:
subscriptions = load_world_bank_data('data/downloads/broadband_cellular_subscriptions/broadband_cellular_subscriptions.csv')

In [ ]:
# View null value matrix
msno.matrix(subscriptions)

In [12]:
# export
subscriptions_long = transform_to_long(subscriptions, id_vars=['Series Name', 'Country Name', 'Country Code'], \
                    var_name='Year', value_name='subs per 100')

export_world_bank_data(subscriptions, 'subscriptions')
export_world_bank_data(subscriptions_long, 'subscriptions_long')

### 3. Internet servers

In [ ]:
# Load internet server data
internet_servers = load_world_bank_data('Data/downloads/secure_internet_servers/secure_internet_servers.csv')


In [ ]:
# view null value matrix
msno.matrix(internet_servers)

In [26]:
# 1990 and 2000 completely empty
internet_servers.drop(['1990', '2000'], axis=1, inplace=True)

In [37]:
# export
internet_servers_long = transform_to_long(internet_servers, id_vars=['Series Name', 'Country Name', 'Country Code'], \
                    var_name='Year', value_name='no. servers')

export_world_bank_data(internet_servers, 'internet_servers')
export_world_bank_data(internet_servers_long, 'internet_servers_long')

### 4. 4g Coverage

Now let's get some data from other sources
- Share of people in range of 4G network: Data sources: International Telecommunication Union – processed by Our World in Data

In [ ]:
# Load data from Our World in Data
four_g_access = pd.read_csv('data\downloads\share-with-4g-mobile-network-coverage\share-with-4g-mobile-network-coverage.csv')
four_g_access.head()

In [42]:
four_g_access.to_csv('4g_access_long.csv')

### 5. Other Economic Indicators

In [ ]:
# Load other standard development indicators
economic_indicators = load_world_bank_data('data\downloads\economic_indicators\economic_indicators.csv')
economic_indicators.head()

In [ ]:
economic_indicators.tail(2)

In [ ]:
# view missing data
msno.matrix(economic_indicators)

In [ ]:
# Import data to Zoho analytics
zoho_client = AnalyticsClient(client_id=, client_secret=, refresh_token=)

In [4]:
help(AnalyticsClient)

## ✍️ Judging criteria
| CATEGORY | WEIGHTING | DETAILS                                                              |
|:---------|:----------|:---------------------------------------------------------------------|
| **Visualizations** | 50% | <ul><li>Appropriateness of visualizations used.</li><li>Clarity of insight from visualizations.</li></ul> |
| **Summary** | 35%       | <ul><li>Clarity of insights - how clear and well presented the findings are.</li>
| **Votes** | 15% | <ul><li>Up voting - most upvoted entries get the most points.</li></ul> |

## 🧾 Executive summary
_In a couple of lines, write your main findings here._

## 📷 Visual/Dashboard screenshot
_Paste one screenshot of your visual/dashboard here._

## 🌐 Upload your dashboard (optional)
Ideally, paste your link to your online available dashboard here.

Otherwise, upload your dashboard file to the _Files_ section (_File_ > _Show workbook files_).

can I have mermaid in a jupyter notebook?

```mermaid
erDiagram
usage
servers
subscriptions
economic_indicators
four_g_coverage
```

